# DatabricksEmbeddings

> [Databricks](https://www.databricks.com/) Lakehouse 平台在一个平台上统一了数据、分析和 AI。

本笔记本提供了使用 Databricks [embedding models](/docs/concepts/embedding_models) 的快速入门指南。有关所有 `DatabricksEmbeddings` 功能和配置的详细文档，请参阅[API 参考](https://python.langchain.com/api_reference/community/embeddings/langchain_community.embeddings.databricks.DatabricksEmbeddings.html)。



## 概述
### 集成详情

| Class | Package |
| :--- | :--- |
| [DatabricksEmbeddings](https://python.langchain.com/api_reference/community/embeddings/langchain_community.embeddings.databricks.DatabricksEmbeddings.html) | [databricks-langchain](https://python.langchain.com/docs/integrations/providers/databricks/) |

### 支持的方法

`DatabricksEmbeddings` 支持 `Embeddings` 类所有方法，包括异步 API。


### Endpoint 要求

`DatabricksEmbeddings` 封装的托管端点必须具有 OpenAI 兼容的embedding 输入/输出格式（[参考](https://mlflow.org/docs/latest/llms/deployments/index.html#embeddings)）。只要输入格式兼容，`DatabricksEmbeddings` 即可用于托管在 [Databricks Model Serving](https://docs.databricks.com/en/machine-learning/model-serving/index.html) 上的任何端点类型：

1. 基础模型 - 目录中包含最先进的基础模型，如 BAAI 通用Embedding (BGE)。这些端点无需设置即可在您的 Databricks 工作区中使用。
2. 自定义模型 - 您还可以通过 MLflow 部署自定义 embedding 模型到托管端点，支持您选择的框架，如 LangChain、Pytorch、Transformers 等。
3. 外部模型 - Databricks 端点可以作为代理托管 Databricks 外部的模型，例如专有模型服务，如 OpenAI text-embedding-3。


## 设置

要访问 Databricks 模型，您需要创建一个 Databricks 帐户，设置凭证（仅当您不在 Databricks 工作区内时），并安装所需的包。

### 凭证（仅当您不在 Databricks 内时）

如果您在 Databricks 工作区内运行 LangChain 应用，则可以跳过此步骤。

否则，您需要手动将 Databricks 工作区主机名和个人访问令牌分别设置为 `DATABRICKS_HOST` 和 `DATABRICKS_TOKEN` 环境变量。有关如何获取访问令牌，请参阅[身份验证文档](https://docs.databricks.com/en/dev-tools/auth/index.html#databricks-personal-access-tokens)。

In [ ]:
import getpass
import os

os.environ["DATABRICKS_HOST"] = "https://your-workspace.cloud.databricks.com"
if "DATABRICKS_TOKEN" not in os.environ:
    os.environ["DATABRICKS_TOKEN"] = getpass.getpass(
        "Enter your Databricks access token: "
    )

### 安装

LangChain Databricks 集成位于 `databricks-langchain` 包：

In [ ]:
%pip install -qU databricks-langchain

## 实例化

In [ ]:
from databricks_langchain import DatabricksEmbeddings

embeddings = DatabricksEmbeddings(
    endpoint="databricks-bge-large-en",
    # Specify parameters for embedding queries and documents if needed
    # query_params={...},
    # document_params={...},
)

## 索引和检索

嵌入模型常用于检索增强生成 (RAG) 流程中，既可以作为索引数据的组成部分，也可以用于之后的检索。有关更详细的说明，请参阅我们的 [RAG 教程](/docs/tutorials/)。

下面将展示如何使用我们上面初始化的 `embeddings` 对象来索引和检索数据。在本例中，我们将在 `InMemoryVectorStore` 中索引和检索一个示例文档。

In [ ]:
# Create a vector store with a sample text
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# Use the vectorstore as a retriever
retriever = vectorstore.as_retriever()

# Retrieve the most similar text
retrieved_document = retriever.invoke("What is LangChain?")

# show the retrieved document's content
retrieved_document[0].page_content

## 直接使用

在底层，vectorstore 和 retriever 实现会分别调用 `embeddings.embed_documents(...)` 和 `embeddings.embed_query(...)` 来为 `from_texts` 和检索 `invoke` 操作中使用的文本创建 embedding。

你可以直接调用这些方法来为自己的用例获取 embedding。

### 嵌入单个文本

你可以使用 `embed_query` 来嵌入单个文本或文档：

In [ ]:
single_vector = embeddings.embed_query(text)
print(str(single_vector)[:100])  # Show the first 100 characters of the vector

### 嵌入多个文本

您可以使用 `embed_documents` 来嵌入多个文本：

In [ ]:
text2 = (
    "LangGraph is a library for building stateful, multi-actor applications with LLMs"
)
two_vectors = embeddings.embed_documents([text, text2])
for vector in two_vectors:
    print(str(vector)[:100])  # Show the first 100 characters of the vector

### 异步用法

您还可以使用 `aembed_query` 和 `aembed_documents` 来异步生成嵌入：

In [ ]:
import asyncio


async def async_example():
    single_vector = await embeddings.aembed_query(text)
    print(str(single_vector)[:100])  # Show the first 100 characters of the vector


asyncio.run(async_example())

## API 参考

有关 `DatabricksEmbeddings` 功能和配置选项的详细文档，请参阅 [API 参考](https://python.langchain.com/api_reference/community/embeddings/langchain_community.embeddings.databricks.DatabricksEmbeddings.html)。